In [ ]:
!pip -q install  openai tiktoken
%pip install git+https://github.com/hwchase17/langchain

In [2]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [3]:
!pip show langchain

Name: langchain
Version: 0.0.237
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: C:\Users\freestone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: aiohttp, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Classification / Tagging


In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from enum import Enum
from langchain.chains.openai_functions import (
    create_tagging_chain,
    create_tagging_chain_pydantic,
)


In [11]:
class PersonalDetails(BaseModel):
    # 定义数据的类型
    name: str = Field(
        ...,
        description = "这是用户输入的名字"
    )
    city: str = Field(
        ...,
        description = "这是用户输入的居住城市"
    )
    email: str = Field(
        ...,
        description = "这是用户输入的邮箱地址"
    )

In [7]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [22]:
chain = create_tagging_chain_pydantic(PersonalDetails,llm)

In [184]:
test_str1 = "你好，我是美丽，我住在上海浦东，我的邮箱是： liteli1987@gmail.com"
test_res1 = chain.run(test_str1)
test_res1

PersonalDetails(name='美丽', city='上海浦东', email='liteli1987@gmail.com')

第二个测试，我只告诉她我的邮箱。机器人只记录了邮箱。

In [174]:
test_str2 = "我的邮箱是： liteli1987@gmail.com"
test_res2 = chain.run(test_str2)
test_res2

PersonalDetails(name='', city='', email='liteli1987@gmail.com')

我们可以再来第三个测试，不告诉机器人我的名字，但是告诉他邮箱，而且还故意告诉他我弟弟的邮箱。

In [177]:
test_str3 = "我叫美丽，我弟弟的邮箱是：1106968391@qq.com"
test_res3 = chain.run(test_str3)
test_res3

PersonalDetails(name='美丽', city='', email='')

我们可以看到结果是'', 如果没有匹配我们定义的PersonalDetails对象里的数据类型，机器人并不会瞎编乱造。

In [35]:
user_007_personal_details = PersonalDetails(name="",city="",email="")

In [36]:
user_007_personal_details

PersonalDetails(name='', city='', email='')

定义一个函数，用于检查数据是否填写完整。

In [37]:
def check_what_is_empty(user_personal_details):
    ask_for = []
    # 检查项目是否为空
    for field,value in user_personal_details.dict().items():
        if value in [None, "", 0]: 
            print(f"Field '{field}' 为空" )
            ask_for.append(f'{field}')
    return ask_for        

我们来测试一下用户007是否填写完整了。

In [38]:
ask_for = check_what_is_empty(user_007_personal_details)
ask_for

Field 'name' 为空
Field 'city' 为空
Field 'email' 为空


['name', 'city', 'email']

我们再来定义一个函数，用于获取用户的输入信息并且更新用户的信息。

In [180]:
def add_non_empty_details(current_details:PersonalDetails, new_details:PersonalDetails):
    # 这是已经填好的用户信息
    non_empty_details = {k:v for k,v in new_details.dict().items() if v not in [None, "", 0]}
    update_details = current_details.copy(update=non_empty_details)
    return update_details

In [181]:
res = chain.run("我的名字007") 
user_007_personal_details = add_non_empty_details(user_007_personal_details,res)
user_007_personal_details

PersonalDetails(name='007', city='南京', email='XX@qq.com')

In [149]:
res = chain.run("我住在南京") 
user_007_personal_details = add_non_empty_details(user_007_personal_details,res)
user_007_personal_details

PersonalDetails(name='007', city='南京', email='XX@qq.com')

In [150]:
res = chain.run("我的邮箱是XX@qq.com") 
user_007_personal_details = add_non_empty_details(user_007_personal_details,res)
user_007_personal_details

PersonalDetails(name='007', city='南京', email='XX@qq.com')

测试一下哪一项没有填写

In [182]:
ask_for = check_what_is_empty(user_007_personal_details)
ask_for

[]

In [152]:
if not ask_for:
    print("谢谢您的回答！我没有问题了")

谢谢您的回答！我没有问题了


使用自定义提示模板，实现机器人发起提问。

In [61]:
def ask_for_info(ask_for=["name","city","email"]):
    # 定义一个提示模板
    first_prompt = ChatPromptTemplate.from_template(
        """
        假设你现在是一名前台，你现在需要对用户进行询问他个人的具体信息。
        不要跟用户打招呼！你可以解释你需要什么信息。不要说“你好！”！
        接下来你和用户之间的对话都是你来提问，凡是你说的都是问句。
        你每次随机选择{ask_for}列表中的一个项目，向用户提问。
        比如["name","city"]列表，你可以随机选择一个"name", 你的问题就是“请问你的名字是？”
        """
    )
    info_gathering_chain = LLMChain(llm=llm, prompt=first_prompt)
    chat_chain = info_gathering_chain.run(ask_for=ask_for)
    return chat_chain
    

In [183]:
ask_for_info(ask_for=["name","city","email"])

'请问你的名字是？'

定义一个处理模型返回信息的函数。

In [104]:
def filter_response(text_input, user_details):
    # 我们使用到openAI提供的打标签链, 用户的输入可以被这个链解析和映射。
    chain = create_tagging_chain_pydantic(PersonalDetails,llm)
    res = chain.run(text_input)
    # 更新用户信息
    user_details = add_non_empty_details(user_details,res)
    ask_for = check_what_is_empty(user_details)
    return user_details, ask_for

测试一下效果，我们要获取的就是最新的用户信息，筛选出来哪些项目还没有填写。


In [155]:
def decide_ask(ask_for=["name","city","email"]):
    if ask_for:
        ai_res = ask_for_info(ask_for=ask_for)
        print(ai_res)
    else:
        print("全部填写完整")
decide_ask(ask_for)        

请问你的名字是？


In [167]:
user_999_personal_details = PersonalDetails(name="",city="",email="")
user_999_personal_details

PersonalDetails(name='', city='', email='')

In [168]:
decide_ask(ask_for)

请问你的名字是？


In [169]:
str999 = "我的名字是999,我住在北京"

In [170]:
user_999_personal_details, ask_for_999 = filter_response(str999,user_999_personal_details)
decide_ask(ask_for_999)

Field 'email' 为空
请问您的电子邮件地址是多少？


In [173]:
str999 = "XX@XX.com"
user_999_personal_details, ask_for_999 = filter_response(str999,user_999_personal_details)
decide_ask(ask_for_999)


全部填写完整
